**Initial Setup**

Server setup and initial values

In [ ]:
import time
import krpc
from math import e
import numpy as np
import matplotlib.pyplot as plt

# Server
conn = krpc.connect(name='Launch into orbit')
vessel = conn.space_center.active_vessel

# Set Up Streams for telemetry
ut = conn.add_stream(getattr, conn.space_center, 'ut')
altitude = conn.add_stream(getattr, vessel.flight(), 'mean_altitude')
q = conn.add_stream(getattr, vessel.flight(), 'dynamic_pressure')
apoapsis = conn.add_stream(getattr, vessel.orbit, 'apoapsis_altitude')
stage_9_resources = vessel.resources_in_decouple_stage(stage=9, cumulative=False)
srb_fuel = conn.add_stream(stage_9_resources.amount, 'SolidFuel')

# Initial Values
turn_start_altitude = 250
turn_end_altitude = 45000
target_altitude = 150000

# Constants
for name, obj in conn.space_center.bodies.items():
    if name == 'Kerbin':
        kerbin = obj
    elif name == 'Mun':
        mun = obj

mass_kerbin = 5.2915158e22  # (kg)
G = 6.67408e-11  # (m**3/kg/s**2) gravitational constant
# mu = G*mass_kerbin
# print(mu,kerbin.gravitational_parameter)
mu = kerbin.gravitational_parameter


**Helper Functions**

In [ ]:
def execute_node(conn, vessel, node, burn_duration):
    while conn.space_center.ut < (node.ut-(burn_duration/2)):
        time.sleep(0.1)

    while node.remaining_delta_v > 15:
        vessel.control.throttle = 1
    while node.remaining_delta_v > 0.1:
        vessel.control.throttle = 0.05

    node.remove()
    vessel.control.throttle = 0

**Orbital Mechanics**

In [ ]:
def calc_burn_duration(vessel, dV):
    ''' Calculates the burn time for a given vessel and a given delta-V. '''
    m = vessel.mass
    isp = vessel.specific_impulse
    thrust = vessel.available_thrust
    term1 = (m*isp*9.81)/(thrust)
    term2 = (1-e**(dV/(isp*9.81)))
    print(thrust)
    return abs(term1*term2)

def calc_circ_orb_speed(r):
    ''' Calculates the orbital speed of a vessel in a circular orbit. '''
    return np.sqrt(mu/r)

def vis_viva(r, a):
    '''
    Calculates the orbital speed of a vessel at some point in an
    elliptical orbit given the semi-major axis and mass of orbiting
    body.
    '''
    return np.sqrt(mu*((2/r)-(1/a)))

**Pre-Launch**

In [ ]:

vessel.control.sas = False
vessel.control.rcs = False
vessel.control.throttle = 0.75

# Countdown...
print('10...')
time.sleep(1)
print('9...')
time.sleep(1)
print('8...')
time.sleep(1)
print('7...')
time.sleep(1)
print('6...')
time.sleep(1)
print('5...')
time.sleep(1)
print('4...')
time.sleep(1)
print('3...')
time.sleep(1)
print('2...')
time.sleep(1)
print('1...')
time.sleep(1)
print('Liftoff!')

**Ascent Phase**

In [ ]:
# Activate Main Engines
vessel.control.activate_next_stage()
vessel.auto_pilot.engage()
vessel.auto_pilot.target_pitch_and_heading(90, 0)
time.sleep(1)

# Active SRBs and decouple umbilicals
vessel.control.activate_next_stage()

# Tower Clear
while altitude() < 200:
    pass
print('Tower Clear, Begin Roll Porgram')
# Roll Program
vessel.auto_pilot.target_pitch_and_heading(90, 90)

# Main ascent loop
srbs_separated = False
roll_program = False
turn_angle = 0
while True:

    # Gravity turn
    if altitude() > turn_start_altitude and altitude() < turn_end_altitude:
        frac = ((altitude() - turn_start_altitude) /
                (turn_end_altitude - turn_start_altitude))
        new_turn_angle = np.sqrt(frac) * 90
        if abs(new_turn_angle - turn_angle) > 0.5:
            turn_angle = new_turn_angle
            vessel.auto_pilot.target_pitch_and_heading(90-turn_angle,90)
        if not roll_program:
            if vessel.auto_pilot.heading_error < 10:
                roll_program = True
                print('Roll Program Complete')

    # Separate SRBs when finished
    if not srbs_separated:
        if srb_fuel() < 40:
            vessel.control.activate_next_stage()
            srbs_separated = True
            print('SRBs separated')

    # Decrease throttle when approaching target apoapsis
    if apoapsis() > target_altitude*0.9:
        print('Approaching target apoapsis')
        break


# Disable engines when target apoapsis is reached
vessel.control.throttle = 0.25
while apoapsis() < target_altitude:
    pass
print('Target apoapsis reached')
vessel.control.throttle = 0.0
print('MECO')
time.sleep(1)
vessel.control.activate_next_stage()
time.sleep(5)

**Post-Ascent Coast**

In [ ]:
# Jettison LAS
vessel.control.activate_next_stage()
print('LAS Jettisonned')

# Wait until out of atmosphere
print('Coasting out of atmosphere')
while altitude() < 70500:
    pass

**PRM Planning**

In [ ]:
# Plan PRM (using vis-viva equation)
print('Planning circularization burn')
vessel.control.activate_next_stage()
a = vessel.orbit.semi_major_axis
r = vessel.orbit.apoapsis
v1 = vis_viva(r,a)
v2 = calc_circ_orb_speed(r)
delta_v = v2 - v1
node = vessel.control.add_node(
    ut() + vessel.orbit.time_to_apoapsis, prograde=delta_v)

# Calculate burn time (using rocket equation)
burn_time = calc_burn_duration(vessel,delta_v)
print(burn_time)

In [ ]:
# Orientate ship
print('Orientating ship for circularization burn')
vessel.control.sas = True
vessel.control.sas_mode = conn.space_center.SASMode.maneuver
vessel.auto_pilot.reference_frame = node.reference_frame
vessel.auto_pilot.target_direction = (0, 1, 0)
vessel.auto_pilot.wait()
time.sleep(1)
ang_vel = vessel.angular_velocity(vessel.orbit.body.reference_frame)
oriented = all(ang_vel[i] < 0.01 for i in range(3))
while not oriented:
    ang_vel = vessel.angular_velocity(vessel.orbit.body.reference_frame)
    oriented = all(ang_vel[i] < 0.01 for i in range(3))

# Wait until burn
print('Waiting until circularization burn')
burn_ut = ut() + vessel.orbit.time_to_apoapsis - (burn_time/2.)
lead_time = burn_time/2.
conn.space_center.warp_to(burn_ut - lead_time)

**MEB2 (PRM)**

In [ ]:
# Execute burn
print('Ready to execute burn')
time_to_apoapsis = conn.add_stream(getattr, vessel.orbit, 'time_to_apoapsis')
while time_to_apoapsis() - (burn_time/2.) > 0:
    pass
print('Executing burn')
vessel.control.throttle = 1.0
vessel.auto_pilot.target_direction = (0, 1, 0)
print(vessel.thrust)
while node.remaining_delta_v > 10:
    pass
print('Fine tuning')
vessel.control.throttle = 0.05
remaining_burn = conn.add_stream(node.remaining_burn_vector, node.reference_frame)
while remaining_burn()[1] > 0.1:
    pass
vessel.control.throttle = 0.0
node.remove()

print('PRM Compelete')
print('Launch complete')